# Black-Scholes-Merton model


In [ ]:
import torch
import matplotlib.pyplot as plt
plt.style.use("dark_background")
%matplotlib inline
torch.manual_seed(987654);

## 1 Constant volatility
This model describes the evolution the forward $f_t$ via GBM dynamics:

$$
\text{d}f_t = \mu f_t\text{d}t + \sigma f_t \text{d}W_t
$$

The volatility $\sigma$ is called the *implied volatility* and is provided by market participants. Being constant is a severe limitation for practitioners, but we will address this issue with stochastic volatility models later.

Calibrating the drift $\mu$ is not possible without apriori information which fundamentally depends on each investors view on the market. 

The igenious idea was to note that the choice of $\mu$ is linked to the choice of probability measure $\mathbb{P}$.

### 1.1 Girsanov Theorem

We will discuss only a special case. Consider a process $\gamma_t$ with finite Ito norm $\int \gamma^2_t dt < \infty$. Associate to $\gamma_t$ the exponential process:
$$
\text{d}Z^{\gamma}_t = - Z_t \gamma_t \text{d}W_t, \quad Z_0=1
$$

We can use this process to define a new probability measure equivalent to $\mathbb{P}$:
$$
\text{d}\mathbb{Q}^\gamma = Z_T^{\gamma}\text{d}\mathbb{P}
$$

Girsanov showed that the process:
$$
\text{d}W^\gamma_t = \gamma_t\text{d}t + \text{d}W_t, \quad W^\gamma_0=0
$$
is a Wiener process under the measure $\mathbb{Q}^\gamma$.

### 1.2 Existence of Risk-Neutral Measure

In the BSM model, substituting:
$$
\gamma = \frac{\mu}{\sigma}
$$
into Girsanov's result leads to martingale dynamics for the forward:
$$
\text{d}f_t = \sigma f_t \text{d}W^\gamma_t
$$
under $\mathbb{Q}^\gamma$. To avoid arbitrage, the forward values of all derivative contracts must also be martingales under that measure. 

Does the valuation we obtained in the risk neutral world is the fair price for the actual measure $\mathcal{P}$?

### 1.3 Black-Scholes PDE

Let's start again from the BSM model. Considering European contracts for simplicity, the  Feynman-Kac formula for the risk-neutral dynamics gives:
$$
\frac{\partial FV}{\partial t} + \frac{1}{2}\sigma^2 f_t^2 \frac{\partial^2 FV}{\partial t^2}=0
$$

This is also known as the *Black-Scholes PDE*.

### 1.4 Replication

By Ito and the above, under the measure $\mathbb{P}$, the derivative's $FV_t$ evolves as:
$$
\begin{split}
\text{d}FV_t & = \left(\frac{\partial FV}{\partial t}+ \mu f_t \frac{\partial FV}{\partial f_t} + \frac{1}{2}\sigma^2 f_t^2 \frac{\partial^2 FV}{\partial t^2} \right)\text{d}t + \sigma f_t\frac{\partial FV}{\partial f_t} dW_t \\
 & =  \mu f_t \frac{\partial FV}{\partial f_t}\text{d}t   + \sigma f_t\frac{\partial FV}{\partial f_t} dW_t \\
 & = \frac{\partial FV}{\partial f_t} df_t
\end{split}
$$

and so we can exactly replicate the derivative by trading in the forward on the underlying as dictated by $\partial FV / \partial f_t$ - the so called *delta*. 

By no-arbitrage principle, we conclude that the risk neutral price is fair for *any* measure $\mathbb{P}$ within the framework of the BSM model.

### 1.6 Black-Scholes formulae

Under the BSM model:
$$
\frac{\text{d}f_t}{f_t} = \sigma \text{d}W_t
$$

The forward value of a European Call at strike $K$ and maturity date $T$ is given by:
$$
FV_C = f_t \text{N}(d_{+}) - K \text{N}(d_{-})
$$

where $N(x) = \frac{1}{\sqrt{2\pi}}\int^x_{-\infty} \text{d}z \exp(-z^2/2)$ is the standard normal CDF and:
$$
d_{\pm} = \frac{\log \frac{f_t}{K} \pm \frac{1}{2}\sigma^2(T-t) }{\sigma \sqrt{T-t}  } 
$$


For the European Put, one can rely on the put-call parity $FV_C - FV_P = f_t - K$:
$$
FV_P = -f_t \text{N}(-d_{+}) + K \text{N}(- d_{-})
$$

#### 1.6.1 Greeks

There are three main risk sensitivies:
* *Delta* $\Delta$ measures the exposure to the underlying:
$$
\begin{split}
 \Delta & \equiv \frac{\partial FV_C}{\partial f} = \text{N}(d_{+})   \\
 \Delta & \equiv \frac{\partial FV_P}{\partial f} = - \text{N}(-d_{+}) = \text{N}(d_{+}) - 1
\end{split}
$$

* *Vega* (also often called *Kappa*) is the exposure to volatility:
$$
\mathcal{V} \equiv \frac{\partial FV_C}{\partial \sigma} = \frac{\partial FV_P}{\partial \sigma} = \text{N}'(d_{+}) f_t \sqrt{T-t}
$$
* *Curvature* is supposed to measure the convexity:
$$
\Upsilon \equiv \frac{\sigma}{\left(f_t\sigma \sqrt{T-t}\right)^2} \mathcal{V} = \frac{\text{N}'(d_{+})}{f_t\sigma \sqrt{T-t}}
$$

The definition of curvature above is somewhat of an abuse of terminology as we should have been capturing second order information with *Gamma*:
$$
\Gamma \equiv \frac{\partial^2 FV_C}{\partial f^2} = \frac{\partial^2 FV_P}{\partial f^2} = \frac{\text{N}'(d_{+})}{f_t\sigma \sqrt{T-t}}
$$

But higher order derivatives are expensive to compute. And as we can easily see, in the special case of BSM this is justified via what is known the *Gamma-Vega relation*.

Finally, another quantity of interest is *Theta* - the decay of the value of optionality over time:
$$
 \Theta \equiv \frac{\partial FV_C}{\partial t}= \frac{\partial FV_P}{\partial t} = - \frac{\sigma f_t \text{N}'(d_{+})}{2 \sqrt{T-t}}  
$$

## References

* Seydel, R.U. *Tools for Computational Finance*, 6th edition, Springer V., London (2017)
* Shreve, S.E. Stochastic Calculus for Finance I & II, Springer, New York (2004)
* Grzelak, L.A. et al. *Mathematical Modeling and Computation in Finance: With Exercises and Python and MATLAB Computer Codes*, World Scientific Pub. (2019)
* Savine, A. Modern Computational Finance: AAD and Parallel Simulations, Wiley (2018)